## Pole figures at DanMAX  
The following notebook is part of the development of texture analysis at DanMAX, including pole figure analysis.
The current implementation relies on user-defined peaks and indices and uses peak integration (trapezoidal integration) to determine the intensities.  
A better approach would be to estimate peak indices and positions from a .cif file and fit the intensities from a Le Bail fit. The [pyobjcryst](https://github.com/diffpy/pyobjcryst) project might be a way to accomplish that.  

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binned_statistic_2d
from scipy.signal import find_peaks
#To import DanMAX from the folder above:
import sys
sys.path.append('../')
import DanMAX as DM
from DanMAX import texture as tx
style = DM.darkMode(style_dic={'figure.figsize':'large'})

### DanMAX coordinate definition  
The coordinates are here defined from eulerian angles: ω χ, and φ, using YXY rotations. The laboratory coordinates X<sub>L</sub>, Y<sub>L</sub>, and Z<sub>L</sub> are defined such that Z<sub>L</sub> is along the beam,  Y<sub>L</sub> is vertical with the positive direction upwards and  X<sub>L</sub> is horizontal, pointing away from the ring. Consequently, ω is a rotation in the horizontal plane around Y<sub>L</sub>, χ is a rotation around the new X<sub>L</sub>', and φ is a rotation around Y<sub>L</sub>''.  
Example 1:  
A sample normal along the beam has the (x,y,z) coordinates (0,0,1). Rotating ω 90° would result in the new coordinates (x',y',z') = (1,0,0).  
Example 2:  
A sample normal along the beam rotated along all three angles 90° results in the following new corrdinates: (0,0,1) -> (1,0,0) -> (0,-1,0) -> (0,0,-1)  
  
NB: while the sample normal in example 2 ends up anti-parallel with the beam, it is *not* the same as a 180° ω-rotation as the sample will also have rotated around the normal. 



In [ ]:
# DanMAX laboratory coordinates:
# For ω=χ=φ=0:
#   X=XL, Y=YL, and Z=ZL
# For ω=Ψ=φ=0 and χ=90:
#   X=XL, Y=ZL, and Z=-YL

# sample orientation
chi_fix   = 0.  # degrees
phi_fix   = 0.  # degrees

In [ ]:
fname = DM.findScan(636)
aname = DM.getAzintFname(fname)

# read azimuthally binned data
data = DM.getAzintData(aname)

# read common meta data from the master file
meta = DM.getMetaData(fname,custom_keys={'omega':'entry/instrument/theta/value'})

# reduce time-resolution to speed up initial analysis
rf = 1
start = None 
end =  None
data = DM.reduceDic(data,reduction_factor=rf,start=start,end=end)
meta = DM.reduceDic(meta,reduction_factor=rf,start=start,end=end)


tth = data['tth']
tth_edge = data['tth_edge']
I = data['I']
cake = data['cake']
azi = data['azi']
azi_edge = data['azi_edge']
t = meta['time'] # relative time stamp in seconds
T = meta['temp'] # temperature in Kelvin (if available, otherwise None)
I0 = meta['I0']  # relative incident beam intensity "I zero"
E = meta['energy'] # X-ray energy in keV
omega = meta['omega']

I_avg = np.mean(I,axis=0)
cake[cake<=0]= np.nan

# estimate omega boundaries
omega_edge = np.append(omega-np.mean(np.diff(omega))/2,omega[-1]+np.mean(np.diff(omega))/2)


chi = np.repeat(chi_fix,len(omega))
phi = np.repeat(phi_fix,len(omega))
chi_edge = np.repeat(chi_fix,len(omega_edge))
phi_edge = np.repeat(phi_fix,len(omega_edge))


In [ ]:
#              hkl  :   ROI 
reflections = {'111' : [ 8.2,  9.05],
               '200' : [ 9.7, 10.2 ],
               '202' : [13.8, 14.4 ],
               '311' : [16.2, 16.8 ],
               '222' : [16.9, 17.6 ],
               '400' : [19.7, 20.3 ]}


# initialize figure
fig = plt.figure()

plt.plot(tth, I_avg,'k-',label='average pattern')
for hkl in reflections:
    roi = reflections[hkl]
    roi = (tth>roi[0]) & (tth<roi[1])
    
    # plot average diffraction pattern and heatmap
    plt.plot(tth[roi],I_avg[roi],'.',ms=3,label=hkl)
plt.legend()
plt.xlabel(r'$2\theta (°)$')
plt.yticks([])
plt.yscale('log')

#### Estimate peak positions and regions of interest
Try to guess peaks and peak positions and output a simple list for copy-pasting (Tip: press down `alt` to enable columnwise cursor selection)  
It might be necessary to tweak the `find_peaks` *prominence* and *wlen* parameters

In [ ]:
#              hkl  :   ROI 
reflections = {'111' : [ 8.04,  9.21],
               '200' : [ 9.37, 10.55],
               '202' : [13.52, 14.68],
               '311' : [15.99, 17.03],
               '222' : [17.03, 17.85],
               '400' : [19.52, 20.58],
               '331' : [21.23, 22.09],
               '420' : [22.09, 22.95],
               '422' : [23.99, 25.15],
               '511' : [25.61, 26.65],
               '440' : [27.87, 29.00],
               '531' : [29.21, 30.04],
               '442' : [30.04, 30.74],
               '620' : [31.45, 32.36],
               '533' : [32.72, 33.25],
               '622' : [33.25, 34.06]}


peaks, prop = find_peaks((I_avg-I_avg.min())/(I_avg.max()-I_avg.min())*100,
                         prominence=0.5,
                         wlen=100)
# correct for overlap
prop['right_bases'][:-1] = np.array([min(prop['left_bases'][i+1],prop['right_bases'][i]) for i in range(len(peaks)-1)])

roi_bgr = tth != tth
# initialize figure
fig = plt.figure()
plt.plot(tth, I_avg,'k.-',ms=1.5,lw=1,label='average pattern')
print(' #  hkl  :    ROI (°)')
for i,peak in enumerate(peaks):
    #plt.plot(tth[peak],y_mean[peak],'o')
    
    roi = tth[prop['left_bases'][i]],tth[prop['right_bases'][i]]
    print(f"{i+1:>2d} '???' : [{roi[0]:>5.2f}, {roi[1]:>5.2f}],")
    roi = (tth>=roi[0]) & (tth<roi[1])
    roi_bgr += roi # region of interest for background points (inverted)
    # plot average diffraction pattern and heatmap
    color = plt.plot(tth[roi], I_avg[roi],'.',ms=3,label=i)[-1].get_color()
    plt.annotate(f'#{i+1}',(tth[peak],I_avg[peak]+1),color=color)
    
roi_bgr = ~roi_bgr    
#plt.legend()
plt.xlabel(r'$2\theta (°)$')
plt.yticks([])
plt.yscale('log')

#### Calculate and plot partial pole figures (fixed resolution)
The integrated points are binned to be equidistant in pole coordinates


In [ ]:
# pole figure resolution
PF_resolution = 1.
subtract_background = False

# Set the number of columns for the figure
cols = 4

rows = int(len(reflections)/cols) + (len(reflections)%cols!=0)
# initialize figure
fig, axes = plt.subplots(rows,cols,subplot_kw={'projection': 'polar'})
axes = axes.flatten()
for ax in axes:
    ax.set_theta_direction(-1)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_rlim(0,pi/2)
    ax.grid(False)

pcm = [] # pcolormeshes
for k,hkl in enumerate(reflections):

    roi = (tth>reflections[hkl][0]) & (tth<reflections[hkl][1])

    # Modification to np.trapz to better handle nan values
    # nan values are (assumed) invariant along the sample rotation axis (omega)
    # and only change along the azimuthal and radial axes
    y = np.full(cake.shape[:2],np.nan)
    # loop through the azimuthal axis
    for j in range(cake.shape[1]):
        yi = cake[:,j,roi]
        if subtract_background:
            bgr = (np.nanmean(yi[:,:3],axis=1)+np.nanmean(yi[:,-3:],axis=1))/2
        else:
            bgr = 0
        # if a sufficient number of points are non-nan, estimate the integral of the peaks along the 2theta axis
        if np.count_nonzero(np.isnan(yi[0]))<(len(yi[0])*0.10):
            y[:,j] = np.trapz(yi[:,~np.isnan(yi[0])] - bgr,
                              x=tth[roi][~np.isnan(yi[0])],
                              axis=1) 
    # set non-physical (negative) values to nan  
    y[y<0]=np.nan
    
    # scattering vector in lab coord
    Q = tx.Q_unit(eta,np.mean(tth[roi]))

    # allocate an empty array
    Q_s = np.full((3,len(eta),len(omega)),
                  0.)
    # iterate through the 3x3xm rotation matrix
    for i,R in enumerate(tx.R_yxy(omega, chi, phi).T):
        # linear transformation from lab-basis to sample-basis
        Q_s[:,:,i] =  np.matmul(R,Q)

    rad, azi = tx.poleAngles(Q_s)

    # Bin integrated intensities to be equidistant in pole coordinates
    rad_bins, azi_bins = [np.arange(0,180+PF_resolution,PF_resolution), 
                          np.arange(0,360+PF_resolution,PF_resolution)]
    
    res = binned_statistic_2d(rad.flatten(),
                              azi.flatten(),
                              y.T.flatten(),
                              statistic= np.mean,  # NOTE Could this be replaced by np.trapz ?
                              bins=[rad_bins,azi_bins],
                              expand_binnumbers=True)
    y_bin = res.statistic
    y_bin[y_bin<=0]=np.nan
    rad_bin_edge, azi_bin_edge = res.x_edge, res.y_edge
    rad_bin = rad_bin_edge[:-1]+PF_resolution/2
    azi_bin = azi_bin_edge[:-1]+PF_resolution/2
    # determine global vmin and vmax
    if k>0:
        vmin,vmax = min(vmin,np.nanmin(y_bin)), max(vmax,np.nanmax(y_bin))
    else:
        vmin,vmax = np.nanmin(y_bin) ,np.nanmax(y_bin)
    
    # plot pole figure
    ax = axes[k]
    ax.set_title(f'({hkl})')
    pcm.append(ax.pcolormesh(azi_bin_edge*pi/180,
                        rad_bin_edge*pi/180,
                        y_bin))
    
    save_txt = False
    if save_txt:
        header = f'DanMAX partial pole figure\nResolution: {PF_resolution:.2f} deg\nradial azimuthal intensity'
        dst = DM.getScan_id(fname)+f'_partialPF_{hkl}.txt'
        nan_mask = ~np.isnan(y_bin.flatten())
        stacked_data = np.stack([np.repeat(rad_bin,len(azi_bin))[nan_mask], # 0,0,0 ... 180,180,180
                                 np.tile(azi_bin,len(rad_bin))[nan_mask],   # 0,1,2 ... 178,179,180
                                 y_bin.flatten()[nan_mask]]).T
        
        np.savetxt(dst,
                   stacked_data,
                   delimiter=' ',
                   header=header,
                   fmt='%.2f')
    
# Toggle global colorscale limits
# (generally not relevant for partial pole figures, 
#  as they are not properly normalized)
share_clim = False
if share_clim:
    for p in pcm:
        p.set_clim(vmin,vmax)
else:
    for i,p in enumerate(pcm):
        fig.colorbar(p,
                     ax=axes[i],
                     fraction=0.05,
                     shrink=0.75)
#fig.colorbar(p,label='a.u.')

# delete surplus plots
for i in range(1,cols*rows-len(reflections)+1):
    fig.delaxes(axes[-i])
    
    

#### Calculate and plot partial pole figures (resolution as-measured)
Use the measured bin-edges to plot every measured point. Mainly used as a sanity-check

In [ ]:
subtract_background = True

# Set the number of columns for the figure
cols = 3

rows = int(len(reflections)/cols) + (len(reflections)%cols!=0)
# initialize figure
fig, axes = plt.subplots(rows,cols,subplot_kw={'projection': 'polar'})
axes = axes.flatten()
for ax in axes:
    ax.set_theta_direction(-1)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_rlim(0,pi/2)
    ax.grid(False)

pcm = [] # pcolormeshes
for k,hkl in enumerate(reflections):

    roi = (tth>reflections[hkl][0]) & (tth<reflections[hkl][1])

    # Modification to np.trapz to better handle nan values
    # nan values are (assumed) invariant along the sample rotation axis (omega)
    # and only change along the azimuthal and radial axes
    y = np.full(cake.shape[:2],np.nan)
    # loop through the azimuthal axis
    for j in range(cake.shape[1]):
        yi = cake[:,j,roi]
        if subtract_background:
            bgr = (np.nanmean(yi[:,:3],axis=1)+np.nanmean(yi[:,-3:],axis=1))/2
        else:
            bgr = 0
        # if a sufficient number of points are non-nan, estimate the integral of the peaks along the 2theta axis
        if np.count_nonzero(np.isnan(yi[0]))<(len(yi[0])*0.10):
            y[:,j] = np.trapz(yi[:,~np.isnan(yi[0])]- bgr,
                              x=tth[roi][~np.isnan(yi[0])],
                              axis=1) 
    # set non-physical (negative) values to nan  
    y[y<0]=np.nan
    
    roi = (tth_edge>reflections[hkl][0]) & (tth_edge<reflections[hkl][1])
    
    # scattering vector in lab coord
    Q = tx.Q_unit(eta_edge,np.mean(tth_edge[roi]))

    # allocate an empty array
    Q_s = np.full((3,len(eta_edge),len(omega_edge)),
                  0.)
    # iterate through the 3x3xm rotation matrix
    for i,R in enumerate(tx.R_yxy(omega_edge, chi_edge, phi_edge).T):
        # linear transformation from lab-basis to sample-basis
        Q_s[:,:,i] =  np.matmul(R,Q)

    rad_edge, azi_edge = tx.poleAngles(Q_s)
    
    if k>0:
        #print(vmin,vmax)
        y /= y_max/100
        vmin,vmax = min(vmin,np.nanmin(y)), max(vmax,np.nanmax(y))
        #print(vmin,vmax)
    else:
        y_max = np.nanmax(y)
        y /= y_max/100
        vmin,vmax = np.nanmin(y) ,np.nanmax(y)
    
    # plot pole figure
    ax = axes[k]
    ax.set_title(f'({hkl})')
    pcm.append(ax.pcolormesh(azi_edge*pi/180,
                        rad_edge*pi/180,
                        y.T))
    

share_clim = False
if share_clim:
    for p in pcm:
        p.set_clim(vmin,vmax)
else:
    for i,p in enumerate(pcm):
        fig.colorbar(p,
                     ax=axes[i],
                     fraction=0.05,
                     shrink=0.75)
#fig.colorbar(p,label='a.u.')

# delete surplus plots
for i in range(1,cols*rows-len(reflections)+1):
    fig.delaxes(axes[-i])